<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train4_rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:

train4 = pd.read_csv(r'/content/train_data4.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [ ]:

train4['Target'] = train4['Target'].replace(['Others'],'Invalid')


In [ ]:
train4['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [ ]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train4_Xtfidf = tfidf_vect.fit_transform(train4['Sentence'])


In [ ]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [ ]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [ ]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [ ]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [ ]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [ ]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train4_Xtfidf,train4['Target'],test_size=0.26,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.665929203539823
                  precision    recall  f1-score   support

        Analysis       0.50      0.35      0.41       127
      Conclusion       0.74      0.40      0.52        42
           Facts       0.73      0.86      0.79       487
         Invalid       0.51      0.58      0.54       128
           Issue       0.59      0.47      0.52        49
Rule/Law/Holding       0.71      0.34      0.46        71

        accuracy                           0.67       904
       macro avg       0.63      0.50      0.54       904
    weighted avg       0.66      0.67      0.65       904



In [ ]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [ ]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 0, 3, 3, 2, 4, 3, 0, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 3, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 1, 3, 3, 2, 2, 2, 2, 2, 2, 3, 5, 2, 2, 2, 3, 2]
[0.9983021958425319, 0.9982806642636489, 0.9934690898635297, 0.9987354269340737, 0.9988232542035954, 0.9837587476895958, 0.9985712418311398, 0.9832668651560081, 0.9981955211543915, 0.9949806491235289, 0.9966020970880862, 0.9976832574160791, 0.9959409533063316, 0.9962950625320284, 0.9946168562565442, 0.998181862152708, 0.9983005918543679, 0.9985073536356956, 0.9817234833250456, 0.9980160313414899, 0.9977346500810909, 0.9898972753785715, 0.9971312188254023, 0.99212481994679, 0.9954712599075903, 0.9970315666494993, 0.9948417774969112, 0.9815530167024137, 0.9931346833246456, 0.9940200768377705, 0.9836744075202137, 0.9950434076245677, 0.9970561717904728, 0.9876581862162119, 0.9945359523166089, 0.9976660200693784, 0.9852558773952944, 0.9965451519157581, 0.9851209395802639, 0.9986200014239782, 0.9940508987172

In [ ]:
unlabel_1=unlabel_1.loc[pos,:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
0,2173,darden sr testified âœi saw staggering â prett...,Facts
1,466,additional moral justification may due complai...,Analysis
2,525,85 22 sw 140 wherein accused acting upon advic...,Rule/Law/Holding
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts


In [ ]:
frame_1 = [train4,unlabel_1]
train4_1 = pd.concat(frame_1)
len(train4_1)

3538

In [ ]:
x_train_1 = tfidf_vect.transform(train4_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train4_1['Target'],test_size=0.2,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6567796610169492
                  precision    recall  f1-score   support

        Analysis       0.44      0.28      0.34        98
      Conclusion       0.67      0.45      0.54        31
           Facts       0.72      0.86      0.78       380
         Invalid       0.55      0.56      0.56       110
           Issue       0.68      0.42      0.52        40
Rule/Law/Holding       0.51      0.39      0.44        49

        accuracy                           0.66       708
       macro avg       0.60      0.49      0.53       708
    weighted avg       0.64      0.66      0.64       708



In [ ]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3, 2, 3, 1, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 2, 2, 2, 2, 2, 2, 5, 3, 2]
[0.9986242543340229, 0.9990355697215404, 0.9941300757167447, 0.9927293867274408, 0.9979623750795475, 0.9983578410481985, 0.9978984168147212, 0.9988419391184293, 0.988719936217974, 0.9801694302829208, 0.9855197678599891, 0.9924369753431559, 0.9942539185798001, 0.9941674567075386, 0.9901419995505284, 0.9983138958029062, 0.9953307919953476, 0.9989760691091282, 0.9835482296950964, 0.9989970488008335, 0.9929444808424478, 0.9988576964912224, 0.9991367709594112, 0.9970517661412224, 0.9968941945130058, 0.9876490081193088, 0.9936452590715302, 0.9988418737238928, 0.997931785768896, 0.9846764271598275, 0.9989606254995396, 0.9990058507504488, 0.9979005307661729, 0.9987899469493425, 0.9955064634080067, 0.9980006128409546, 0.9971345531740284, 0.9913504937901256, 0.9986713467606172, 0.9987316461353181, 0.9849832389691134, 0.9965838070797877, 0.99801799926

In [ ]:
unlabel_2 = unlabel_2.loc[pos,:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
5,2278,evidence state appellant shows appellant alleg...,Analysis
6,570,appellant performed work asked 7500 money ther...,Facts


In [ ]:
frame_2 = [train4_1,unlabel_2]
train4_2 = pd.concat(frame_2)
len(train4_2)

3587

In [ ]:
x_train_2 = tfidf_vect.transform(train4_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train4_2['Target'],test_size=0.27,random_state=2)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6357069143446853
                  precision    recall  f1-score   support

        Analysis       0.48      0.30      0.37       142
      Conclusion       0.69      0.35      0.47        57
           Facts       0.69      0.89      0.78       501
         Invalid       0.52      0.53      0.52       150
           Issue       0.61      0.23      0.34        60
Rule/Law/Holding       0.50      0.24      0.32        59

        accuracy                           0.64       969
       macro avg       0.58      0.42      0.47       969
    weighted avg       0.62      0.64      0.60       969



In [ ]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 3, 2, 2, 2, 3, 3, 2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, 1, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 2, 3, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2]
[0.9979939067710265, 0.9906761999788889, 0.992826880293765, 0.9974565635921127, 0.9965537829093548, 0.9973977251184484, 0.9891227731362899, 0.9982840318685898, 0.9983802250199035, 0.9981615855974652, 0.9836772216417797, 0.9979835352566744, 0.998726963980499, 0.9977511707358672, 0.9984387613812609, 0.9937802498221654, 0.9980744488686745, 0.9857263897106218, 0.9869432054043434, 0.9889908079388574, 0.9970329901280812, 0.9974463913041932, 0.9985567320571888, 0.9918172933163463, 0.9982861680047505, 0.9802531461993177, 0.9836567550566045, 0.9901822148212774, 0.9959662655899353, 0.996709145758528, 0.9980594380499325, 0.9966492202073156, 0.9890579230253252, 0.9932487604945215, 0.9975608144940216, 0.9983240283860939, 0.9897135000593802, 0.9871019873785448, 0.9985216472757198, 0.998092

In [ ]:
unlabel_3=unlabel_3.loc[pos,:]
frame_3 = [train4_2,unlabel_3]
train4_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train4_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train4_3['Target'],test_size=0.2,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6707650273224044
                  precision    recall  f1-score   support

        Analysis       0.45      0.33      0.38        90
      Conclusion       0.68      0.42      0.52        40
           Facts       0.73      0.86      0.79       395
         Invalid       0.56      0.55      0.56       116
           Issue       0.83      0.60      0.70        40
Rule/Law/Holding       0.54      0.29      0.38        51

        accuracy                           0.67       732
       macro avg       0.63      0.51      0.55       732
    weighted avg       0.66      0.67      0.65       732



In [ ]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[3, 0, 3, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 3, 0, 2, 2, 2, 5, 3, 0, 5, 2, 2, 2, 2, 3, 2, 2, 1, 2, 2, 2, 3, 2, 2, 2, 2, 5, 2, 2, 2]
[0.993460131533769, 0.9905129550586774, 0.9915464102295589, 0.9978869030374534, 0.9959376846598672, 0.9980987458656285, 0.9950856135283798, 0.9855689173272765, 0.99884329483195, 0.9988979426658313, 0.9976776006036067, 0.9981167254100587, 0.9932407064513992, 0.9988137517342807, 0.9987621458905607, 0.9983701141884965, 0.9989170827792309, 0.9904943852842188, 0.9978781339063811, 0.9987157364278865, 0.9981107589570033, 0.9988204072195308, 0.997953286539703, 0.9981066285010962, 0.9983619271127734, 0.9986767847998218, 0.9974200710406644, 0.9985817032570117, 0.9965671348659182, 0.9980817237930579, 0.9835903077562065, 0.9873280888452305, 0.9967287264085796, 0.9978022086776014, 0.997270698702961, 0.9828419881491103, 0.9957296100524263, 0.997465176341688, 0.9915577183541336, 0.9912141175224367, 0.9928210179124

In [ ]:
unlabel_4=unlabel_4.loc[pos,:]

In [ ]:
frame_4 = [train4_3,unlabel_4]
train4_4 = pd.concat(frame_4)
len(train4_4)
x_train_4 = tfidf_vect.transform(train4_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train4_4['Target'],test_size=0.27,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6477611940298508
                  precision    recall  f1-score   support

        Analysis       0.47      0.28      0.35       135
      Conclusion       0.67      0.39      0.49        56
           Facts       0.72      0.86      0.79       538
         Invalid       0.50      0.56      0.53       157
           Issue       0.56      0.39      0.46        51
Rule/Law/Holding       0.47      0.26      0.34        68

        accuracy                           0.65      1005
       macro avg       0.57      0.46      0.49      1005
    weighted avg       0.63      0.65      0.63      1005



In [ ]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[3, 2, 3, 3, 2, 3, 4, 2, 5, 3, 3, 2, 2, 3, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 0, 2, 2, 2, 3, 3, 2, 3, 3, 2, 2, 2, 1, 2, 2, 2, 2, 3, 2, 2, 5, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 3, 3, 0, 2, 0, 3, 3, 2]
[0.9867812869730722, 0.9990624710274761, 0.9871988753691102, 0.9990714975076087, 0.9951279889165952, 0.9981319853810667, 0.994974013219843, 0.9982167233156309, 0.9951550108593054, 0.9988740246907972, 0.9990714975076087, 0.9976606427551277, 0.9989917000101499, 0.9876618934314729, 0.9923933917319672, 0.9870278960721316, 0.9950139124082578, 0.9980816786724005, 0.9972900749084617, 0.996220926647299, 0.9990887933298285, 0.9989232402811101, 0.9955581956676742, 0.9866210039426635, 0.9984820761060215, 0.9967415525783009, 0.9952915062608602, 0.9826911871605489, 0.9929259812984986, 0.9981299280087922, 0.9989841850482468, 0.9990309174903842, 0.9990255033945341, 0.9990714975076087, 0.9980868797681196, 0.9943356135212899, 0.9844848260107383, 0.9962554593962142, 0.9989751485324454, 0

In [ ]:
unlabel_5 = unlabel_5.loc[pos,:]

In [ ]:
frame_5 = [train4_4,unlabel_5]
train4_5 = pd.concat(frame_5)
len(train4_5)
x_train_5 = tfidf_vect.transform(train4_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train4_5['Target'],test_size=0.2,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6666666666666666
                  precision    recall  f1-score   support

        Analysis       0.52      0.36      0.43       108
      Conclusion       0.81      0.48      0.60        44
           Facts       0.71      0.88      0.78       393
         Invalid       0.57      0.55      0.56       119
           Issue       0.62      0.33      0.43        45
Rule/Law/Holding       0.62      0.40      0.49        50

        accuracy                           0.67       759
       macro avg       0.64      0.50      0.55       759
    weighted avg       0.66      0.67      0.65       759



In [ ]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.761996161228407


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(test['Target'],test_pred,average='macro'))

(0.7816651573624182, 0.6669198378560611, 0.7098686499932961, None)
